<a href="https://colab.research.google.com/github/saiyuu330/COSE474_Project/blob/main/DL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jaeungyun","key":"2d6b19b1c261f1452c200ad28d1f5208"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d nqa112/vietnamese-bike-and-motorbike

 99% 1.43G/1.44G [00:14<00:00, 94.0MB/s]
100% 1.44G/1.44G [00:14<00:00, 104MB/s] 


In [ ]:
!unzip -o /content/vietnamese-bike-and-motorbike.zip -d /content/data

In [11]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import json
import time
from tqdm import tqdm
from PIL import Image
from struct import unpack
import pathlib

In [9]:
marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}

class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
                break

In [10]:
def delete_corrupted_image(image):
    if str(image).endswith("gif"):
      return image
    else:
      check = JPEG(image)
      try:
        check.decode()
      except:
        return image

In [12]:
data_dir = pathlib.Path("/content/data")

for image in data_dir.glob("*/*"):
  corrupt = delete_corrupted_image(image)
  if corrupt is not None:
    print(f"Delete {image}")
    image.unlink()

Delete /content/data/motorbike/9036559372_d5fd35b32b_o.jpg
Delete /content/data/motorbike/Motor7svrb_zvsmallMotor.jpg
Delete /content/data/motorbike/19_r_636858959614681208_4528_kkk_1568719429934_12092.gif
Delete /content/data/motorbike/96_gia_xe_may_sym_re_nhat_2016_duoc_nguoi_tieu_dung_ua_chuong_4_1568719337886_10799.jpg
Delete /content/data/motorbike/2019_08_05_05_17_32_B0xS_6hHgXG_66398352_483445189138958_8195470045202604419_n_1568719912383_18787.jpg
Delete /content/data/motorbike/79_414925055008472_1568719726002_16213.jpg
Delete /content/data/motorbike/96_201503220449_img_20150202_114557_1568719735476_16338.jpg
Delete /content/data/motorbike/65_51ew43mofl9ef_1568719232558_9326.gif
Delete /content/data/motorbike/69_xedoisong_Yamaha_MT_03_do_1_MEGC_1568719472292_12693.gif
Delete /content/data/motorbike/93_honda_airblade_2014_1568719129105_7903.jpg
Delete /content/data/motorbike/70_201609113434_img20160906150355_1568719720842_16140.jpg
Delete /content/data/motorbike/Motorlccwyb8ksmal

In [13]:
bike_count = len(list(data_dir.glob('bike/*')))
motorbike_count = len(list(data_dir.glob('motorbike/*')))
bike_count, motorbike_count

(6044, 6061)